In [1]:
import pandas as pd
import numpy as np

import clickhouse_connect


client = clickhouse_connect.get_client(
    host='index.bostrom.bronbro.io', port=443,
    username='bronbro', password='spac3b0x')

def query_to_df(query: str) -> pd.DataFrame:
    query_res = client.query(query=query)
    return pd.DataFrame(query_res.result_rows, columns=query_res.column_names)

In [2]:
query_1 = r'''
SELECT *
FROM (SELECT signer,
             groupArrayIf(passport_nickname, passport_nickname != '')                        as passport_nicknames,
             uniqExactIf(proof_address, proof_address != '')                                 as proof_addresses,
             uniqExactIf(remove_address, remove_address != '')                               as remove_addresses,
             uniqExactIf(gift_claiming_address, gift_claiming_address != '')                 as claim_addresses,
             uniqExactIf(release_address, release_address != '')                             as release_addresses,
             round(sum(gift_amount_gboot), 1)                                                as claim_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, '0x')), 1)     as claim_eth_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'cosmos')), 1) as claim_cosmos_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'osmo')), 1)   as claim_osmosis_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'terra')), 1)  as claim_terra_gboot,
             claim_gboot * 0.29                                                              as releasable_gboot,
             round(sum(released_gboot), 1)                                                   as release_gboot
      FROM (SELECT signer,
                   arrayZip(JSONExtractArrayRaw(messages), JSONExtractArrayRaw(raw_log)) as messages_logs_zip,
                   arrayJoin(messages_logs_zip)                                          as message_log_tuple,
                   message_log_tuple.1                                                   as message_row,
                   message_log_tuple.2                                                   as raw_log_item,
                   trim(BOTH '"' FROM
                        replaceAll(replaceAll(message_row, '\\', ''), '@', ''))          as message,
                   JSONExtractString(message, 'type')                                    as message_type,
                   JSONExtractString(message, 'sender')                                  as message_sender,
                   JSONExtractString(message, 'contract')                                as message_contract,
                   JSONExtractString(message, 'msg')                                     as message_msg,
                   JSON_VALUE(message_msg, '$.create_passport.nickname')                 as passport_nickname,
                   JSON_VALUE(message_msg, '$.proof_address.address')                    as proof_address,
                   JSON_VALUE(message_msg, '$.remove_address.address')                   as remove_address,
                   JSON_VALUE(message_msg, '$.release.gift_address')                     as release_address,
                   round(toInt64OrZero(JSON_VALUE(message_msg, '$.claim.gift_amount')) / 1e9,
                         1)                                                              as gift_amount_raw_gboot,
                   JSON_VALUE(message_msg, '$.claim.gift_claiming_address')              as gift_claiming_address,
                   toInt64OrZero(replace(
                           JSON_VALUE(raw_log_item, '$.events[1].attributes[0].value'),
                           'boot', '')) /
                   1e9                                                                   as released_gboot,
                   toInt64OrZero(JSON_VALUE(raw_log_item, '$.events[5].attributes[4].value')) /
                   1e9                                                                   as gift_amount_gboot,
                   date
            FROM (SELECT height,
                         signer,
                         raw_log,
                         messages,
                         date
                  FROM `spacebox`.transaction FINAL
                           LEFT ANY
                           JOIN (
                      SELECT height,
                             toDate(timestamp) as date
                      FROM `spacebox`.block
                          PREWHERE block.height > 10800000
                      ) as b
                                ON transaction.height = b.height
                      PREWHERE transaction.success = true
                      and transaction.height > 10800000
                     )
            WHERE message_type == '/cosmwasm.wasm.v1.MsgExecuteContract'
              and message_contract in ('bostrom1xut80d09q0tgtch8p0z4k5f88d3uvt8cvtzm5h3tu3tsy4jk9xlsfzhxel',
                                       'bostrom16t6tucgcqdmegye6c9ltlkr237z8yfndmasrhvh7ucrfuqaev6xq7cpvek')
            ORDER BY height DESC)
      GROUP BY signer)
-- WHERE release_gboot > 0
ORDER BY release_gboot DESC
'''

gift_by_singer = query_to_df(query_1)

In [3]:
gift_by_singer

,signer,passport_nicknames,proof_addresses,remove_addresses,claim_addresses,release_addresses,claim_gboot,claim_eth_gboot,claim_cosmos_gboot,claim_osmosis_gboot,claim_terra_gboot,releasable_gboot,release_gboot
0,bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6,[etwet46w4],3640,3640,1549,1505,1541.3,1541.3,0.0,0.0,0.0,446.977,434.8
1,bostrom1trt5q36aqdx6l2ca3r5hush2hm9vfu699j9ksn,[fbfbhbvcccccc],2147,2141,2142,2143,1020.5,1020.5,0.0,0.0,0.0,295.945,334.0
2,bostrom1k6kyj0wukla82z2a0ufmrj99xmxhxhj22r04at,[blackhole666],497,497,497,497,450.4,450.4,0.0,0.0,0.0,130.616,153.2
3,bostrom1ulefdrkmfluv5zvhc4nwuw5vtnaspx8sh9wpk3,[proj3proj],1255,1251,1256,1254,391.5,381.7,0.0,3.2,6.5,113.535,131.4
4,bostrom1g4ywagfj8yc03ecjn7ghet5exxa5l4kqwuplv4,[qwrtgwrhwfq],473,473,119,119,366.2,366.2,0.0,0.0,0.0,106.198,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15587,bostrom1ednc63kwckcz6arhzm28f7undd4gqlvygya0ks,[ngnfn554],2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15588,bostrom19jcktgz30hz8vd637cgcxmmqvrut3k0ky68fv3,[54545454545454545454545454542],2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15589,bostrom153cw09k759z52lkmh4xxsgy37gtt6splyf8388,[jasvoyager],0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15590,bostrom1gt4kqr6jsfd987qsgzj8wh7twg8es6ye50qglr,[laloaaaaaaaaa],3,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0


In [4]:
gift_by_singer.groupby('release_addresses').agg({'release_gboot': sum, 'signer': np.count_nonzero, 'release_addresses': sum})

,release_gboot,signer,release_addresses
release_addresses,,,
0,0.0,7905,0
1,5427.5,4434,4434
2,4119.3,1503,3006
3,2442.8,543,1629
4,2253.1,282,1128
5,1549.9,182,910
6,1167.8,163,978
7,702.3,123,861
8,2849.0,415,3320


In [5]:
gift_by_singer.sum()

signer                 bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6...
passport_nicknames     [etwet46w4, fbfbhbvcccccc, blackhole666, proj3...
proof_addresses                                                    58811
remove_addresses                                                   21187
claim_addresses                                                    25141
release_addresses                                                  27067
claim_gboot                                                      78092.5
claim_eth_gboot                                                  68237.3
claim_cosmos_gboot                                                5111.3
claim_osmosis_gboot                                               3928.1
claim_terra_gboot                                                  799.7
releasable_gboot                                               22646.825
release_gboot                                                    22897.7
dtype: object